In [1]:
import pandas as pd

In [2]:
# Read the CSV files into DataFrames
df1 = pd.read_csv('../data/raw/2025-01-10-lisbon-listings-for-rent.csv', index_col='propertyCode')
df2 = pd.read_csv('../data/raw/2025-01-13-lisbon-listings-for-rent.csv', index_col='propertyCode')
df3 = pd.read_csv('../data/raw/2025-01-15-lisbon-listings-for-rent.csv', index_col='propertyCode')

# Combine the DataFrames
df = pd.concat([df1, df2, df3])

# Keep only the last occurrence of each index
df = df.loc[~df.index.duplicated(keep='last')]

# Save the unioned DataFrame to a new CSV file
df.to_csv('../data/raw/unioned-lisbon-listings-for-rent.csv', index_label='propertyCode')

In [3]:
df.index.is_unique

True

In [4]:
df

,thumbnail,numPhotos,floor,price,priceInfo,propertyType,operation,size,rooms,bathrooms,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,topPlus,externalReference,parkingSpace,newDevelopmentFinished
propertyCode,,,,,,,,,,,,,,,,,,,,,
33894270,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,39,NaN,2250.0,"{'price': {'amount': 2250.0, 'currencySuffix':...",flat,rent,375.0,4,3,...,"{'subtitle': 'São João das Lampas e Terrugem, ...",False,False,False,False,False,False,9526HS,NaN,NaN
33894210,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,25,NaN,1620.0,"{'price': {'amount': 1620.0, 'currencySuffix':...",flat,rent,153.0,5,2,...,"{'subtitle': 'Saldanha - Picoas, Lisboa', 'tit...",False,False,False,False,False,False,APA_400,NaN,NaN
33894072,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,24,NaN,1500.0,"{'price': {'amount': 1500.0, 'currencySuffix':...",flat,rent,150.0,3,2,...,"{'subtitle': 'Galiza, Cascais', 'title': 'Apar...",False,False,False,False,False,False,9525JO,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN
33513874,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,20,-2,2700.0,"{'price': {'amount': 2700.0, 'currencySuffix':...",flat,rent,102.0,2,2,...,"{'subtitle': 'Estrela, Lisboa', 'title': 'Apar...",True,False,False,False,False,False,A0363.DV.ES,NaN,NaN
33884590,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,12,3,950.0,"{'price': {'amount': 950.0, 'currencySuffix': ...",flat,rent,121.0,3,2,...,"{'subtitle': 'Santo Onofre e Serra do Bouro, C...",False,False,False,False,False,False,APA_961,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33896870,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,26,NaN,1600.0,"{'price': {'amount': 1600.0, 'currencySuffix':...",flat,rent,82.0,2,2,...,"{'subtitle': 'Lumiar, Lisboa', 'title': 'Apart...",True,False,True,False,False,False,W-02Y9Q5,NaN,NaN
33896861,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,30,NaN,1700.0,"{'price': {'amount': 1700.0, 'currencySuffix':...",flat,rent,70.0,2,1,...,{'subtitle': 'Alameda - Praça do Chile - Almir...,True,False,True,False,False,False,W-02VUYS,NaN,NaN
33896773,https://img4.idealista.pt/blur/WEB_LISTING/0/i...,17,bj,1850.0,"{'price': {'amount': 1850.0, 'currencySuffix':...",flat,rent,126.0,2,2,...,"{'subtitle': 'Lumiar, Lisboa', 'title': 'Apart...",True,False,False,False,False,False,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN
